In [10]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_D1

# Define start date and end date
start_date = datetime(2023, 11, 27, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 9, 25, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print("Failed to retrieve data")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions from CSV files
df_predsell = pd.read_csv('predictadaboostUSDCAD_D1Sell.csv', index_col=0)
df_predsell.index = pd.to_datetime(df_predsell.index)

df_predbuy = pd.read_csv('predictadaboostUSDCAD_D1Buy.csv', index_col=0)
df_predbuy.index = pd.to_datetime(df_predbuy.index)

# Define a simple strategy based on predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0088
    risk_reward_ratiosell = '3:4'
    risk_reward_ratiobuy = '2:3'

    def init(self):
        self.risk_reward_ratiosell = SimpleStrategy.risk_reward_ratiosell
        self.risk_reward_ratiobuy = SimpleStrategy.risk_reward_ratiobuy

    def next(self):
        current_time = self.data.index[-1]
        
        # Get predictions for the current time
        sell_prediction = df_predsell.loc[current_time, 'prediction'] if current_time in df_predsell.index else 0
        buy_prediction = df_predbuy.loc[current_time, 'prediction'] if current_time in df_predbuy.index else 0
        
        # Ensure no trade is placed if both predictions are 1
        if sell_prediction == 1 and buy_prediction == 1:
            return
        
        # Place a sell order if the sell prediction is 1
        if sell_prediction == 1 and buy_prediction != 1:
            entry_price = self.data.Close[-1]
            order_type = 'sell'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiosell, order_type, self.mean_candle_size)
            self.sell(size=100000, sl=sl_price, tp=tp_price)
        
        # Place a buy order if the buy prediction is 1
        if buy_prediction == 1 and sell_prediction != 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiobuy, order_type, self.mean_candle_size)
            self.buy(size=100000, sl=sl_price, tp=tp_price)

# Create and run backtest with the SimpleStrategy
bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
output = bt.run()
print(output)


risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4','2:2','2:3','2:4','2:5']

# Optimize the strategy for SL/TP ratios
stats = bt.optimize(
    risk_reward_ratiosell=risk_reward_ratios,
    risk_reward_ratiobuy=risk_reward_ratios,
    maximize='Equity Final [$]',
    constraint=lambda p: ':' in p.risk_reward_ratiosell and ':' in p.risk_reward_ratiobuy
)

# Print optimized strategy stats
print(f"Best risk_reward_ratiosell: {stats._strategy.risk_reward_ratiosell}")
print(f"Best risk_reward_ratiobuy: {stats._strategy.risk_reward_ratiobuy}")
print(stats)

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Plot the results of the optimized strategy
bt.plot()


Start                     2019-07-12 00:00:00
End                       2021-10-06 00:00:00
Duration                    817 days 00:00:00
Exposure Time [%]                   12.413793
Equity Final [$]                          0.0
Equity Peak [$]                    11690.2271
Return [%]                             -100.0
Buy & Hold Return [%]                7.993226
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]         55697346.651966
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                  -69.645496
Max. Drawdown Duration      743 days 00:00:00
Avg. Drawdown Duration      272 days 00:00:00
# Trades                                   41
Win Rate [%]                         39.02439
Best Trade [%]                       0.984432
Worst Trade [%]                     -0.939109
Avg. Trade [%]                    

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1407', ...)

In [4]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_H4

# Define start date and end date
start_date = datetime(2022, 2, 2, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 6, 22, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print("Failed to retrieve data")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions from CSV files
df_predsell = pd.read_csv('predictadaboostUSDCAD_D1Sell.csv', index_col=0)
df_predsell.index = pd.to_datetime(df_predsell.index)

df_predbuy = pd.read_csv('predictadaboostUSDCAD_D1Buy.csv', index_col=0)
df_predbuy.index = pd.to_datetime(df_predbuy.index)

# Define a simple strategy based on predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0088
    risk_reward_ratiosell = '3:4'
    risk_reward_ratiobuy = '2:3'

    def init(self):
        self.risk_reward_ratiosell = self.risk_reward_ratiosell
        self.risk_reward_ratiobuy = self.risk_reward_ratiobuy

    def next(self):
        current_time = self.data.index[-1]
        
        # Get predictions for the current time
        sell_prediction = df_predsell.loc[current_time, 'prediction'] if current_time in df_predsell.index else 0
        buy_prediction = df_predbuy.loc[current_time, 'prediction'] if current_time in df_predbuy.index else 0
        
        # Ensure no trade is placed if both predictions are 1
        if sell_prediction == 1 and buy_prediction == 1:
            return
        
        # Place a sell order if the sell prediction is 1
        if sell_prediction == 1 and buy_prediction != 1:
            entry_price = self.data.Close[-1]
            order_type = 'sell'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiosell, order_type, self.mean_candle_size)
            self.sell(size=100000, sl=sl_price, tp=tp_price)
        
        # Place a buy order if the buy prediction is 1
        if buy_prediction == 1 and sell_prediction != 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiobuy, order_type, self.mean_candle_size)
            self.buy(size=100000, sl=sl_price, tp=tp_price)

# List of risk-reward ratios to test
risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4', '2:2', '2:3', '2:4', '2:5']

# DataFrame to store results for each combination of risk-reward ratios
results_columns = ['Equity Final [$]', 'Equity Peak [$]', 'Return [%]', 'Max. Drawdown [%]',
                   'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration', 'Win Rate [%]']
results_df = pd.DataFrame(index=pd.MultiIndex.from_product([risk_reward_ratios, risk_reward_ratios], names=['Sell Ratio', 'Buy Ratio']), columns=results_columns)

# Run backtest for each combination of risk-reward ratios
for sell_ratio in risk_reward_ratios:
    for buy_ratio in risk_reward_ratios:
        print(f"Testing risk-reward ratios: sell={sell_ratio}, buy={buy_ratio}")
        bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
        stats = bt.run(risk_reward_ratiosell=sell_ratio, risk_reward_ratiobuy=buy_ratio)
        
        # Store the results in the DataFrame
        results_df.loc[(sell_ratio, buy_ratio), 'Equity Final [$]'] = stats['Equity Final [$]']
        results_df.loc[(sell_ratio, buy_ratio), 'Equity Peak [$]'] = stats['Equity Peak [$]']
        results_df.loc[(sell_ratio, buy_ratio), 'Return [%]'] = stats['Return [%]']
        results_df.loc[(sell_ratio, buy_ratio), 'Max. Drawdown [%]'] = stats['Max. Drawdown [%]']
        results_df.loc[(sell_ratio, buy_ratio), 'Avg. Drawdown [%]'] = stats['Avg. Drawdown [%]']
        results_df.loc[(sell_ratio, buy_ratio), 'Max. Drawdown Duration'] = stats['Max. Drawdown Duration']
        results_df.loc[(sell_ratio, buy_ratio), 'Avg. Drawdown Duration'] = stats['Avg. Drawdown Duration']
        results_df.loc[(sell_ratio, buy_ratio), 'Win Rate [%]'] = stats['Win Rate [%]']

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Print the results matrix
print("\nResults matrix:")
print(results_df)

# Optionally plot the last backtest results
bt.plot()


Testing risk-reward ratios: sell=1:1, buy=1:1
Testing risk-reward ratios: sell=1:1, buy=1:2
Testing risk-reward ratios: sell=1:1, buy=1:3
Testing risk-reward ratios: sell=1:1, buy=1:4
Testing risk-reward ratios: sell=1:1, buy=2:2
Testing risk-reward ratios: sell=1:1, buy=2:3
Testing risk-reward ratios: sell=1:1, buy=2:4
Testing risk-reward ratios: sell=1:1, buy=2:5
Testing risk-reward ratios: sell=1:2, buy=1:1
Testing risk-reward ratios: sell=1:2, buy=1:2
Testing risk-reward ratios: sell=1:2, buy=1:3
Testing risk-reward ratios: sell=1:2, buy=1:4
Testing risk-reward ratios: sell=1:2, buy=2:2
Testing risk-reward ratios: sell=1:2, buy=2:3
Testing risk-reward ratios: sell=1:2, buy=2:4
Testing risk-reward ratios: sell=1:2, buy=2:5
Testing risk-reward ratios: sell=1:3, buy=1:1
Testing risk-reward ratios: sell=1:3, buy=1:2
Testing risk-reward ratios: sell=1:3, buy=1:3
Testing risk-reward ratios: sell=1:3, buy=1:4
Testing risk-reward ratios: sell=1:3, buy=2:2
Testing risk-reward ratios: sell=1

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3089', ...)

In [ ]:
results_df

Equity Final [$] Equity Peak [$]  Return [%]  \
Sell Ratio Buy Ratio                                                
1:1        1:1             17296.1744      19172.6032   72.961744   
           1:2             27451.0725      28564.1703  174.510725   
           1:3             30057.4898      33084.5876  200.574898   
           1:4             39182.4783      43097.5761  291.824783   
           2:2             46736.9281      48026.0259  367.369281   
...                               ...             ...         ...   
2:5        1:4             75487.2923       79204.997  654.872923   
           2:2             85145.8005      92620.5052  751.458005   
           2:3             89958.9347      97236.6394  799.589347   
           2:4             98615.8273     101627.2447  886.158273   
           2:5             95267.2114      97001.6947  852.672114   

                     Max. Drawdown [%] Avg. Drawdown [%]  \
Sell Ratio Buy Ratio                                       
1:1        1:1              -38.490801         -7.866644   
           1:2              -38.479452         -9.367029   
           1:3              -51.914079         -13.73779   
           1:4              -45.505352         -9.686257   
           2:2              -28.513023         -5.088071   
...                                ...               ...   
2:5        1:4              -47.327281         -6.232158   
           2:2              -31.309983         -4.402416   
           2:3              -45.591037         -5.534228   
           2:4              -45.591037         -5.565725   
           2:5               -55.43513         -6.720005   

                     Max. Drawdown Duration Avg. Drawdown Duration  \
Sell Ratio Buy Ratio                                                 
1:1        1:1            197 days 04:00:00       18 days 08:00:00   
           1:2            136 days 08:00:00       15 days 23:00:00   
           1:3            125 days 16:00:00       24 days 02:00:00   
           1:4            125 days 16:00:00       14 days 21:00:00   
           2:2            144 days 16:00:00        9 days 18:00:00   
...                                     ...                    ...   
2:5        1:4            126 days 00:00:00        9 days 05:00:00   
           2:2             88 days 00:00:00        8 days 04:00:00   
           2:3             71 days 00:00:00        9 days 07:00:00   
           2:4             77 days 00:00:00        9 days 16:00:00   
           2:5             71 days 12:00:00        8 days 22:00:00   

                     Win Rate [%]  
Sell Ratio Buy Ratio               
1:1        1:1          59.561129  
           1:2          53.674121  
           1:3          48.709677  
           1:4          48.333333  
           2:2          64.214047  
...                           ...  
2:5        1:4          45.925926  
           2:2          61.172161  
           2:3          56.766917  
           2:4          56.521739  
           2:5          55.982906  

[64 rows x 8 columns]

In [5]:
results_df_sorted = results_df.sort_values(by='Max. Drawdown Duration', ascending=True)


In [6]:
results_df_sorted

Equity Final [$] Equity Peak [$]  Return [%]  \
Sell Ratio Buy Ratio                                                
2:3        2:3            100536.4879     109053.2001  905.364879   
2:2        2:3             87336.4879      95853.2001  773.364879   
2:3        2:2             84522.1112      90025.8234  745.221112   
2:2        2:2             65452.0691      70955.7813  554.520691   
2:4        2:3              87196.621      95713.3332   771.96621   
...                               ...             ...         ...   
1:4        1:2              1084.0408       13517.553  -89.159592   
           1:3               862.2531       13517.553  -91.377469   
           1:4                    0.0       13517.553      -100.0   
1:1        1:2               301.2123      12300.7245  -96.987877   
           1:1              1168.0646      11216.4519  -88.319354   

                     Max. Drawdown [%] Avg. Drawdown [%]  \
Sell Ratio Buy Ratio                                       
2:3        2:3              -70.218649         -9.440711   
2:2        2:3              -76.558079         -9.449748   
2:3        2:2              -65.289481         -7.365546   
2:2        2:2              -77.712027         -9.018314   
2:4        2:3              -90.824738        -13.516215   
...                                ...               ...   
1:4        1:2              -91.980495        -17.573043   
           1:3              -93.621234        -17.682426   
           1:4                  -100.0        -18.107677   
1:1        1:2              -97.551264        -27.179949   
           1:1              -89.586149        -26.473863   

                     Max. Drawdown Duration Avg. Drawdown Duration  \
Sell Ratio Buy Ratio                                                 
2:3        2:3            160 days 00:00:00        7 days 14:00:00   
2:2        2:3            273 days 04:00:00        9 days 08:00:00   
2:3        2:2            278 days 04:00:00        8 days 05:00:00   
2:2        2:2            279 days 08:00:00        9 days 20:00:00   
2:4        2:3            317 days 08:00:00       14 days 06:00:00   
...                                     ...                    ...   
1:4        1:2            497 days 16:00:00       57 days 01:00:00   
           1:3            497 days 16:00:00       57 days 01:00:00   
           1:4            497 days 16:00:00       57 days 01:00:00   
1:1        1:2            497 days 16:00:00      106 days 23:00:00   
           1:1            849 days 08:00:00      213 days 22:00:00   

                     Win Rate [%]  
Sell Ratio Buy Ratio               
2:3        2:3               62.5  
2:2        2:3               62.5  
2:3        2:2          61.481481  
2:2        2:2          60.583942  
2:4        2:3          59.649123  
...                           ...  
1:4        1:2          18.181818  
           1:3          18.518519  
           1:4          18.518519  
1:1        1:2               25.0  
           1:1          40.909091  

[64 rows x 8 columns]

In [ ]:
csv_path = "sorteddf.xlsx"
results_df_sorted.to_excel(csv_path)


PermissionError: [Errno 13] Permission denied: 'sorteddf.xlsx'

In [6]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    else:
        raise ValueError("order_type must be 'buy'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "GBPUSD"
timeframe = mt5.TIMEFRAME_D1

# Define start date and end date
start_date = datetime(2023, 11, 24, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 8, 13, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print("Failed to retrieve data")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions from CSV files
df_predbuy = pd.read_csv('GBPUSD_D1_3112_Buy.csv', index_col=0)
df_predbuy.index = pd.to_datetime(df_predbuy.index)

# Define a simple strategy based on predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.011
    risk_reward_ratiobuy = '1:1'

    def init(self):
        self.risk_reward_ratiobuy = self.risk_reward_ratiobuy

    def next(self):
        current_time = self.data.index[-1]
        
        # Get prediction for the current time
        buy_prediction = df_predbuy.loc[current_time, 'prediction'] if current_time in df_predbuy.index else 0
        
        # Place a buy order if the buy prediction is 1
        if buy_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratiobuy, order_type, self.mean_candle_size)
            self.buy(size=10000, sl=sl_price, tp=tp_price)

# List of risk-reward ratios to test
risk_reward_ratios = ['1:1']

# DataFrame to store results for each combination of risk-reward ratios
results_columns = ['Equity Final [$]', 'Equity Peak [$]', 'Return [%]', 'Max. Drawdown [%]',
                   'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration', 'Win Rate [%]']
results_df = pd.DataFrame(index=risk_reward_ratios, columns=results_columns)

# Run backtest for each risk-reward ratio
for buy_ratio in risk_reward_ratios:
    print(f"Testing risk-reward ratio: buy={buy_ratio}")
    bt = Backtest(df, SimpleStrategy, cash=100000, commission=0.0003, margin=0.01)
    stats = bt.run(risk_reward_ratiobuy=buy_ratio)
    
    # Store the results in the DataFrame
    results_df.loc[buy_ratio, 'Equity Final [$]'] = stats['Equity Final [$]']
    results_df.loc[buy_ratio, 'Equity Peak [$]'] = stats['Equity Peak [$]']
    results_df.loc[buy_ratio, 'Return [%]'] = stats['Return [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown [%]'] = stats['Max. Drawdown [%]']
    results_df.loc[buy_ratio, 'Avg. Drawdown [%]'] = stats['Avg. Drawdown [%]']
    results_df.loc[buy_ratio, 'Max. Drawdown Duration'] = stats['Max. Drawdown Duration']
    results_df.loc[buy_ratio, 'Avg. Drawdown Duration'] = stats['Avg. Drawdown Duration']
    results_df.loc[buy_ratio, 'Win Rate [%]'] = stats['Win Rate [%]']

# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Print the results matrix
print("\nResults matrix:")
print(results_df)

# Optionally plot the last backtest results
bt.plot()


Testing risk-reward ratio: buy=1:1

Results matrix:
    Equity Final [$] Equity Peak [$] Return [%] Max. Drawdown [%]  \
1:1     101586.17915    102373.11067   1.586179         -1.929094   

    Avg. Drawdown [%] Max. Drawdown Duration Avg. Drawdown Duration  \
1:1         -0.691529       80 days 00:00:00       23 days 00:00:00   

    Win Rate [%]  
1:1    54.621849  


c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3930', ...)

In [8]:
results_df_sorted = results_df.sort_values(by='Max. Drawdown Duration', ascending=True)
results_df_sorted

,Equity Final [$],Equity Peak [$],Return [%],Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,Win Rate [%]
2:2,57696.97,63200.6822,476.9697,-81.864732,-19.125599,283 days 12:00:00,13 days 09:00:00,67.46988
1:3,0.0,22563.4873,-100.0,-100.0,-21.23917,420 days 12:00:00,33 days 14:00:00,17.948718
1:4,0.0,25285.1034,-100.0,-100.0,-18.611812,420 days 12:00:00,27 days 23:00:00,14.814815
2:3,0.0,41110.7854,-100.0,-100.0,-16.131473,420 days 12:00:00,22 days 19:00:00,33.333333
2:4,0.0,50873.7843,-100.0,-100.0,-16.808368,420 days 12:00:00,22 days 19:00:00,26.923077
2:5,0.0,54991.7843,-100.0,-100.0,-17.768002,420 days 12:00:00,23 days 22:00:00,24.0
1:1,1092.2045,12981.4764,-89.077955,-91.586439,-14.565496,497 days 16:00:00,55 days 20:00:00,41.304348
1:2,975.4127,14307.4764,-90.245873,-93.182497,-16.201426,497 days 16:00:00,56 days 03:00:00,22.222222
